In [1]:
from matplotlib import image
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from skimage.metrics import structural_similarity
from skimage.morphology import skeletonize
from skimage.util import invert

import numpy as np
from PIL import Image
import cv2
import random
from math import acos

## Тут более упорядочены используемые функции

In [2]:
def draw_picture(image, bgr=False):
    '''Рисует изображение'''
    b, g, r = cv2.split(image) # по умолчанию cv2 почему-то отдает цвета в порядке BGR вместо RGB
    new_image = cv2.merge([r, g, b])
    plt.figure(figsize=(7, 5))
    plt.axis('off')
    plt.imshow(new_image)
    plt.show()
    
def draw_grey_picture(fname, save=False, name='5_cl.png'):
    if type(fname) == type('ft'):
        image = Image.open(fname).convert("L")
    else:
        image = fname
    arr = np.asarray(image)
    plt.imshow(arr, cmap='gray', vmin=0, vmax=255)

    if save:
        plt.savefig(name)
    #plt.show()
    return arr

def cluster(img, algo=KMeans, n_clusters=2, ret=False, eps=5, min_samples=10, name='5_cl'):
    size = img.shape
    img = img.reshape(size[0]*size[1], 3)
    if algo != DBSCAN:
        model = algo(n_clusters=n_clusters)
    else:
        model = DBSCAN(eps=eps, min_samples=min_samples)
    pred_img_clusters = model.fit_predict(img)
    
    new_picture = img.copy()
    
    for i in range(n_clusters):
        new_picture[pred_img_clusters == i] = [np.median(img[pred_img_clusters == i].transpose()[0]), np.median(img[pred_img_clusters == i].transpose()[1]), np.median(img[pred_img_clusters == i].transpose()[2])]
    
    new_picture = new_picture.reshape(size)
    draw_picture(new_picture)
    
    print('SSIM is:')
    print(structural_similarity(img.reshape(size), new_picture, multichannel=True))
    
    if ret:
        return new_picture
    
    
def cluster_grey(img, algo=KMeans, n_clusters=2, ret=False, eps=5, min_samples=5, save=False, name='5_cl.jpg', make_plot=True):
    size = img.shape
    img = img.reshape(size[0]*size[1]).reshape(-1, 1)
    if algo != DBSCAN:
        model = algo(n_clusters=n_clusters)
    else:
        model = DBSCAN(eps=eps, min_samples=min_samples)
    pred_img_clusters = model.fit_predict(img)
    
    new_picture = img.copy()
    
    for i in range(n_clusters):
        new_picture[pred_img_clusters == i] = np.median(img[pred_img_clusters == i].transpose()[0])
    new_picture = new_picture.reshape(size)
    
    if make_plot:
        draw_grey_picture(Image.fromarray(new_picture), save=save, name=name)

    if ret:
        return new_picture

In [3]:
# карта градиентов
    
def vertical_gradient(im, c=1):
    """Функция принимает на вход grayscale array и параметр c - то, через сколько пикселей считается вертикальный градиент"""
    size = im.shape
    ar = np.zeros((size[0]-c, size[1]))
    for j in range(size[1]):
        for i in range(size[0]-c):
            ar[i][j] = abs(im[i+c][j]-im[i][j])
    return ar

def horizontal_gradient(im, c=1):
    """Функция принимает на вход grayscale array и параметр c - то, через сколько пикселей считается горизонтальный градиент"""
    size = im.shape
    ar = np.zeros((size[0], size[1]-c))
    for i in range(size[1]-c):
        for j in range(size[0]):
            ar[j][i] = abs(im[j][i+c]-im[j][i])
    return ar
def split_by_c(p, c=70):
    """Функция нужна для красивого отображения точек, которые мы хотим взять в выборку для построения прямой"""
    p_c = p.copy()
    p_c[p_c <= c] = 0
    p_c[p_c > c] = 1
    return p_c

def return_train_from_ar_gradient(ar_grad, gr=70):
    """Возвращает по заданной границе точки, которые пойдут для построения прямых"""
    ar = []
    for i in range(ar_grad.shape[0]):
        for j in range(ar_grad.shape[1]):
            if ar_grad[i][j] > gr:
                ar.append([i, j])
    p = np.array(ar)
    return p

def coefs(p, model='simple'):
    """Возвращает коэффициенты прямой, у которой минимальна сумма квадратов расстояний от точек выборки до прямой"""
    x = p[:, 0]
    y = p[:, 1]

    x_ = []
    for i in range(len(x)):
        x_.append([x[i]])
    if model == 'simple':
        model = LinearRegression()
    else:
        model = BayesianRidge()
    model.fit(x_, y)
    return model.coef_[0], model.intercept_

def sum_gradient(im, c=1):
    """Функция принимает на вход grayscale array и параметр c - то, через сколько пикселей считается сумму градиентов"""
    size = im.shape
    ar = np.zeros((size[0]-c, size[1]-c))
    for i in range(size[0]-c):
        for j in range(size[1]-c):
            ar[i][j] = abs(im[i+c][j]-im[i][j]) + abs(im[i][j+c]-im[i][j])
    return ar

In [4]:
def sum_gradient_grey(img, c=4, sum_gr=75, save=False, name='5_cl.jpg', make_plot=True, ret=False):
    
    img = img.astype(int)
    points_sum = sum_gradient(img, c=c)
    data_sum = return_train_from_ar_gradient(points_sum, sum_gr)

    new_sum_pic = np.full(img.shape, 255).astype('uint8')
    for elem in data_sum:
        new_sum_pic[elem[0]][elem[1]] = 0
    new_sum_pic = cv2.medianBlur(new_sum_pic, 3)
    
    if make_plot:
        draw_grey_picture(Image.fromarray(new_sum_pic), save=save, name=name)

    if ret:
        return new_sum_pic

In [5]:
#скелетонизация

def make_boundaries(new_pic_1):

    c = 'white'
    new_pic_1[0][0] = 255

    for i in range(1, new_pic_1.shape[0] - 1):
        if (new_pic_1[i][0] == 0) and new_pic_1[i+1][0] != 0:
            if c == 'grey':
                c = 'white'
            else:
                c = 'grey'
        if c == 'grey':        
            new_pic_1[i][0] = 0


    new_pic_1[new_pic_1.shape[0] - 1][0] = 255
    c = 'white'

    for i in range(1, new_pic_1.shape[1] - 1):
        if new_pic_1[new_pic_1.shape[0] - 1][i] == 0 and new_pic_1[new_pic_1.shape[0] - 1][i+1] != 0:
            if c == 'grey':
                c = 'white'
            else:
                c = 'grey'
        if c == 'grey':        
            new_pic_1[new_pic_1.shape[0] - 1][i] = 0


    new_pic_1[0][new_pic_1.shape[1] - 1] = 255
    c = 'white'

    for i in range(1, new_pic_1.shape[0] - 1):
        if new_pic_1[i][new_pic_1.shape[1] - 1] == 0 and new_pic_1[i+1][new_pic_1.shape[1] - 1] != 0:
            if c == 'grey':
                c = 'white'
            else:
                c = 'grey'
        if c == 'grey':        
            new_pic_1[i][new_pic_1.shape[1] - 1] = 0


    c = 'white'
    new_pic_1[0][new_pic_1.shape[1] - 1] = 255

    for i in range(1, new_pic_1.shape[1] - 1):
        if new_pic_1[0][i] == 0 and new_pic_1[0][i+1] != 0:
            if c == 'grey':
                c = 'white'
            else:
                c = 'grey'
        if c == 'grey':        
            new_pic_1[0][i] = 0 
            
    return new_pic_1



def fat_boundaries(img, algo='cluster', n_clusters=3, c=4, sum_gr=50, save=False, name='3_clusters.jpg'):
    plt.figure(figsize=(20, 20))
    if algo == 'cluster':
        cl_3 = cluster_grey(img=img, algo=KMeans, n_clusters=n_clusters, save=save, name=name, ret=True, make_plot=False)

        grey_num = sorted(set(cl_3.reshape(1, cl_3.shape[0]*cl_3.shape[1]).tolist()[0]))[1]

        cl_3[cl_3 != grey_num] = 255
        cl_3[cl_3 == grey_num] = 0
        
        cl_3 = cv2.medianBlur(cl_3, 5)
        cl_3 = make_boundaries(cl_3)
    else:
        cl_3 = sum_gradient_grey(img=img, c=c, sum_gr=sum_gr, ret=True)
        cl_3 = cv2.medianBlur(cl_3, 3)
        cl_3 = make_boundaries(cl_3)
    
    pic_to_find_points = draw_grey_picture(Image.fromarray(cl_3), save=save, name='grey_'+name)
    
    
    pic_to_find_points = pic_to_find_points.copy()
    
    pic_to_find_points[pic_to_find_points == 0] = 1
    pic_to_find_points[pic_to_find_points == 255] = 0
     
    skeleton = skeletonize(pic_to_find_points)
    
    skeleton[skeleton == True] = 255
    skeleton[skeleton == False] = 0
    
    plt.imshow(skeleton, cmap=plt.cm.gray)
    plt.savefig('bound_' + name)
    
    return pic_to_find_points

In [6]:
#алгоритм упрощения границ

import math
   
def simplify_points (pts, tolerance): 
    anchor  = 0
    floater = len(pts) - 1
    stack   = []
    keep    = set()

    stack.append((anchor, floater))  
    while stack:
        anchor, floater = stack.pop()
      
        # инициализация отрезка
        if pts[floater] != pts[anchor]:
            anchorX = float(pts[floater][0] - pts[anchor][0])
            anchorY = float(pts[floater][1] - pts[anchor][1])
            seg_len = math.sqrt(anchorX ** 2 + anchorY ** 2)
            # get the unit vector
            anchorX /= seg_len
            anchorY /= seg_len
        else:
            anchorX = anchorY = seg_len = 0.0
    
        # внутренний цикл:
        max_dist = 0.0
        farthest = anchor + 1
        for i in range(anchor + 1, floater):
            dist_to_seg = 0.0
            # compare to anchor
            vecX = float(pts[i][0] - pts[anchor][0])
            vecY = float(pts[i][1] - pts[anchor][1])
            seg_len = math.sqrt( vecX ** 2 + vecY ** 2 )
            # dot product:
            proj = vecX * anchorX + vecY * anchorY
            if proj < 0.0:
                dist_to_seg = seg_len
            else: 
                # compare to floater
                vecX = float(pts[i][0] - pts[floater][0])
                vecY = float(pts[i][1] - pts[floater][1])
                seg_len = math.sqrt( vecX ** 2 + vecY ** 2 )
                # dot product:
                proj = vecX * (-anchorX) + vecY * (-anchorY)
                if proj < 0.0:
                    dist_to_seg = seg_len
                else:  # расстояние от до прямой по теореме Пифагора:
                    dist_to_seg = math.sqrt(abs(seg_len ** 2 - proj ** 2))
                if max_dist < dist_to_seg:
                    max_dist = dist_to_seg
                    farthest = i

        if max_dist <= tolerance: # использование отрезка
            keep.add(anchor)
            keep.add(floater)
        else:
            stack.append((anchor, farthest))
            stack.append((farthest, floater))

    keep = list(keep)
    keep.sort()
    return [pts[i] for i in keep]

In [7]:
#информация по углам
def contr_i_to_line(contours):
    contours = contours.tolist()
    contours = list(map(lambda x: tuple(x[0]), contours))
    return contours
def obratno(line):
    line = map(lambda x: [list(x)], line)
    return np.array(list(line))


def leng(ba):
    return (ba[0]**2 + ba[1]**2)**(1/2)

def count_angle(ba, bc):
    c = (ba[0]*bc[0] + ba[1]*bc[1])/(leng(ba) * leng(bc))
    
    if c > 1:
        c = 1
    if c < -1:
        c = -1
        
    c = acos(c)*57.2958
    if ba[0]*bc[1] - ba[1]*bc[0] >= 0:
        return c
    return 360 - c

def angs_sorted(p):
    angs = []
    i = 0
    for i in range(len(p) - 2):
        ba = (p[i][0] - p[i+1][0], p[i][1] - p[i+1][1])
        bc = (p[i+2][0] - p[i+1][0], p[i+2][1] - p[i+1][1])
        ang = count_angle(ba, bc)
        angs.append(ang)
        
    if len(p) > 2:
        ba = (p[i+1][0] - p[i+2][0], p[i+1][1] - p[i+2][1])
        bc = (p[0][0] - p[i+2][0], p[0][1] - p[i+2][1])
        ang = count_angle(ba, bc)
        angs.append(ang)

        ba = (p[i+2][0] - p[0][0], p[i+2][1] - p[0][1])
        bc = (p[1][0] - p[0][0], p[1][1] - p[0][1])
        ang = count_angle(ba, bc)
        angs.append(ang)
    return angs

In [46]:
#функция которая все делает с n-ой картинкой
def learn_everything(num = 1, algo='cluster', ret_dist=False, ret_all=False, show_colors=True, show_simple=True, show_dist=True):
    num = str(num)
    c = 3 - len(num)
    num = c*'0' + num
    img_big_1 = draw_grey_picture('examples/Ultra_Co6_2-' + num + '.png')
    img_big_1 = cluster_grey(img=img_big_1, algo=KMeans, n_clusters=3, ret=True)

    p_1 = fat_boundaries(img=img_big_1, algo=algo)

    # выделим пустоты
    p = p_1
    contours, hierarchy = cv2.findContours(p, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    drawing = np.zeros((p.shape[0], p.shape[1], 3), dtype=np.uint8)
    for i in range(len(contours)):
        color = (random.randint(0,256), random.randint(0,256), random.randint(0,256))
        cv2.drawContours(drawing, contours, i, color, 2, cv2.LINE_8, hierarchy, 0)
    
    # Show in a window
    if show_colors:
        plt.figure(figsize=(20, 20))

        plt.imshow(drawing)

        plt.savefig('examples/done/' + num + '_colors.png')



    # упрощаем

    contours, hierarchy = cv2.findContours(p, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    drawing = np.zeros((p.shape[0], p.shape[1], 3), dtype=np.uint8)

    distribution = [0]*361

    for i in range(len(contours)):
        contours[i] = obratno(simplify_points(contr_i_to_line(contours[i]), 3.0))
        color = (random.randint(0,256), random.randint(0,256), random.randint(0,256))
        cv2.drawContours(drawing, contours, i, color, 2, cv2.LINE_8, hierarchy, 0)
        p = contr_i_to_line(contours[i])
        angs = angs_sorted(p)
        for angle in angs:
            distribution[round(angle)] += 1
    # Show in a window
    if show_simple:
        plt.figure(figsize=(20, 20))

        plt.imshow(drawing)

        plt.savefig('examples/done/' + num + '_simple.png')



    #распределение
    if show_dist:
        plt.figure(figsize=(10, 10))
        plt.bar(range(361), distribution)
        plt.savefig('examples/done/' + num + '_distribution.png')
    
    if ret_all:
        return drawing, distribution
    
    if ret_dist:
        return distribution